In [1]:
import numpy as np
import pandas as pd
import math
import copy
import random
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem import WordNetLemmatizer
import operator
import json
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, EmotionOptions

In [2]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ajinkya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
df = pd.read_csv('tweets1.csv',delimiter=',')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459 entries, 0 to 458
Data columns (total 7 columns):
Date               459 non-null object
Tweet Full Text    459 non-null object
Topic              459 non-null object
Action             459 non-null object
user id            356 non-null float64
follower count     356 non-null float64
place              0 non-null float64
dtypes: float64(3), object(4)
memory usage: 25.2+ KB


In [4]:
topics = df['Topic'].unique()
df['Topic'].value_counts()

Community Action    100
Traffic              93
Appreciation         77
Suggestion           52
Undetermined         31
Follow up            22
Dissatisfaction      20
Query                17
Fraud                13
General Info         13
Theft                 6
Fake News             5
CyberCrime            3
Question              2
Query                 2
Cyber                 1
Others                1
Missing Person        1
Name: Topic, dtype: int64

In [5]:
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in text]
lemmatizer = WordNetLemmatizer()

In [6]:
text = "i am Good Good good person".split()
text = lemmatize_text(text)
vectorizer = CountVectorizer()
print(vectorizer.fit_transform(text).toarray().sum(axis=0))
vectorizer.vocabulary_

[1 3 1]


{'am': 0, 'good': 1, 'person': 2}

In [7]:
text = df['Tweet Full Text'].str.cat(sep=' ').split()
text = lemmatize_text(text)
vectorizer = CountVectorizer()
vectorizer.fit_transform(text)
# print(vectorizer.vocabulary_.keys())
table = pd.DataFrame(index=topics,columns=vectorizer.vocabulary_.keys())
table = table.fillna(0)

In [8]:
for i in list(vectorizer.vocabulary_.keys()):
    if(i=='good'):
        print(i)

good


In [9]:
for x in topics:
    data = df[df['Topic']==x]
    text = data['Tweet Full Text'].str.cat(sep=' ').split()
    text = lemmatize_text(text)
    vectorizer = CountVectorizer()
    count = vectorizer.fit_transform(text).toarray().sum(axis=0)
    name = list(vectorizer.vocabulary_.keys())
    for i in range(len(name)):
        table.loc[x][name[i]] += count[i]
table = table/table.sum()
for c in table:
    table[c] = table[c] * math.log( len(table)/len(table[table[c]!=0]))
table

,zomatocare,hydcitypolice,cyberabadpolice,vijaygopal_,nzomato,accepted,my,order,and,never,...,hqs,knew,ignored,appropriate,cscpcmxoie,raokavitha,needed,deploy,stations,ibtdrbyxeg
Undetermined,2.890372,0.0,0.018430,0.255966,2.890372,2.890372,0.019308,0.199084,0.052357,0.89588,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Follow up,0.000000,0.0,0.018430,0.255966,0.000000,0.000000,0.019308,0.000000,0.010471,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Traffic,0.000000,0.0,0.018430,1.279828,0.000000,0.000000,0.077231,0.000000,0.010471,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Appreciation,0.000000,0.0,0.018430,0.000000,0.000000,0.000000,0.115847,0.000000,0.010471,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
CyberCrime,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.019308,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Question,0.000000,0.0,0.018430,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Others,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Community Action,0.000000,0.0,0.036860,0.000000,0.000000,0.000000,0.038616,0.000000,0.073300,0.44794,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.890372,2.890372,2.890372,2.890372,2.890372
Cyber,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010471,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Query,0.000000,0.0,0.018430,0.000000,0.000000,0.000000,0.019308,0.000000,0.010471,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [38]:
def predict(text):
    global table
    text = text.lower().replace("'",'').replace('!','').replace('.',' ').split()
    text = lemmatize_text(text)
    tp = {}
    for i in text:
        try:
                try:
                    tp[table[i].idxmax()] += max(table[i])
                except:
                    tp[table[i].idxmax()] = max(table[i])
        except:
            a='f'
    print(tp,text)
    if(bool(tp) == False):
        return 'Others'
    return  max(tp.items(), key=operator.itemgetter(1))[0]

In [40]:
predict("b'@hydcitypolice @DeepRK94 Mobile phone driving is too dangerous'")

{'Suggestion': 0.5489716480551704, 'Community Action': 0.8828840900266, 'Traffic': 0.5972531564093516, 'Follow up': 0.027348233519093188, 'Dissatisfaction': 1.464816384890813} ['b@hydcitypolice', '@deeprk94', 'mobile', 'phone', 'driving', 'is', 'too', 'dangerous']


'Dissatisfaction'

In [35]:
df['predictions'] = df['Tweet Full Text'].apply(predict)
df["classification_correct"] = df["predictions"] == df["Topic"]
df['classification_correct'].mean()

0.9084967320261438

In [13]:
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2018-11-16',
    iam_apikey='TQHgzc_QZ5w9hjhnYdFkWN0lrJk4AaXB4PkJXltCL-fA',
    url='https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
)

In [26]:
sadness,joy,fear,disgust,anger = [],[],[],[],[]
for i in df['Tweet Full Text']:
    response2 = natural_language_understanding.analyze(
        text=i,
        features=Features(emotion=EmotionOptions()),
        language='en'
    ).get_result()
    mydict = json.loads(json.dumps(response2['emotion']['document']['emotion']))
    sadness.append(mydict['sadness'])
    joy.append(mydict['joy'])
    fear.append(mydict['fear'])
    disgust.append(mydict['disgust'])
    anger.append(mydict['anger'])
df['sadness'] = sadness
df['joy'] = joy
df['fear'] = fear
df['disgust'] = disgust
df['anger'] = anger
df.head()

,Date,Tweet Full Text,Topic,Action,user id,follower count,place,predictions,classification_correct,sadness,joy,fear,disgust,anger
0,2/25/19 6:34,b'@zomatocare @hydcitypolice \n@cyberabadpolic...,Undetermined,NS,1.080000e+18,0.0,NaN,Undetermined,True,0.105245,0.225655,0.001491,0.002787,0.023300
1,2/25/19 5:15,"b'@hydcitypolice Sir, any updates regarding my...",Follow up,S,1.040000e+18,0.0,NaN,Follow up,True,0.162827,0.033009,0.013693,0.125967,0.187487
2,2/24/19 5:59,b'@HYDTP @CPHydCity @hydcitypolice @TelanganaD...,Traffic,NS,1.090000e+18,0.0,NaN,Traffic,True,0.141880,0.146321,0.088611,0.105356,0.080799
3,03-01-2019 14:32,b'@HYDTP @hydcitypolice @AddlCPTrHyd Good init...,Appreciation,NS,1.400108e+08,0.0,NaN,Appreciation,True,0.054764,0.780943,0.050431,0.014236,0.026168
4,03-01-2019 12:34,b'@USCGHyderabad @TelanganaDGP @USAndHyderabad...,Appreciation,NS,1.050000e+18,0.0,NaN,Appreciation,True,0.002762,0.835799,0.001891,0.001063,0.012618


In [36]:
df

,Date,Tweet Full Text,Topic,Action,user id,follower count,place,predictions,classification_correct,sadness,joy,fear,disgust,anger
0,2/25/19 6:34,b'@zomatocare @hydcitypolice \n@cyberabadpolic...,Undetermined,NS,1.080000e+18,0.0,NaN,Undetermined,True,0.105245,0.225655,0.001491,0.002787,0.023300
1,2/25/19 5:15,"b'@hydcitypolice Sir, any updates regarding my...",Follow up,S,1.040000e+18,0.0,NaN,Follow up,True,0.162827,0.033009,0.013693,0.125967,0.187487
2,2/24/19 5:59,b'@HYDTP @CPHydCity @hydcitypolice @TelanganaD...,Traffic,NS,1.090000e+18,0.0,NaN,Traffic,True,0.141880,0.146321,0.088611,0.105356,0.080799
3,03-01-2019 14:32,b'@HYDTP @hydcitypolice @AddlCPTrHyd Good init...,Appreciation,NS,1.400108e+08,0.0,NaN,Appreciation,True,0.054764,0.780943,0.050431,0.014236,0.026168
4,03-01-2019 12:34,b'@USCGHyderabad @TelanganaDGP @USAndHyderabad...,Appreciation,NS,1.050000e+18,0.0,NaN,Appreciation,True,0.002762,0.835799,0.001891,0.001063,0.012618
5,03-01-2019 08:03,"b""@hydcitypolice @CyberCrimeshyd @HDFC_Bank @H...",CyberCrime,NS,7.530000e+17,0.0,NaN,CyberCrime,True,0.486733,0.037393,0.014823,0.043937,0.072907
6,2/27/19 5:51,b'@hydcitypolice Why are Hyderabad Police too ...,Question,NS,8.060000e+17,0.0,NaN,Question,True,0.050856,0.004335,0.948223,0.035517,0.060600
7,2/27/19 4:48,b'@hydcitypolice Plz take action',Others,NS,1.070000e+18,0.0,NaN,Traffic,False,0.060297,0.081585,0.089984,0.322446,0.093865
8,2/27/19 4:47,b'@hydcitypolice Same way should take Action o...,Community Action,NS,1.070000e+18,0.0,NaN,Community Action,True,0.076255,0.076837,0.103986,0.496847,0.427549
9,2/26/19 13:40,"b'@hydcitypolice Sir/Madam, Please take actio...",Cyber,NS,1.090000e+18,0.0,NaN,Cyber,True,0.441921,0.035492,0.116284,0.183267,0.443134
